<a href="https://colab.research.google.com/github/Prasmaker/Quora-Spam-Filter-using-simpleLSTM-and-GloVe-embedding/blob/main/quora_spam_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

In [2]:
datafile=pd.read_csv(r'/content/drive/MyDrive/quora questions data/train (1).csv',sep=',', encoding='latin-1')

In [64]:
datafile.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [5]:
datafile.isnull().sum()

,0
qid,0
question_text,0
target,0


In [66]:
datafile['question_text'].value_counts()

,count
question_text,
Who wins in a battle between a Wolverine and a Puma?,1
How did Quebec nationalists see their province as a nation in the 1960s?,1
"Do you have an adopted dog, how would you encourage people to adopt and not shop?",1
Why does velocity affect time? Does velocity affect space geometry?,1
How did Otto von Guericke used the Magdeburg hemispheres?,1
...,...
"I am 30, living at home and have no boyfriend. I would love a boyfriend and my own home. How can I progress my situation?",1
Can we use our external hard disk as a OS as well as for data storage.will the data be affected?,1
"What is the dumbest, yet possibly true explanation for Trump being elected?",1


In [67]:
datafile['target'].value_counts()

,count
target,
0,1225312
1,80810


In [12]:
datafile.isnull().sum()


,0
qid,0
question_text,0
target,0


In [ ]:
# Dropping qid since its of no use
datafile.drop(['qid'],axis=1,inplace=True)
datafile.dtypes

In [20]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Downloading GloVe Embeddings

In [21]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2025-06-10 15:16:45--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2025-06-10 15:16:45--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2025-06-10 15:16:46--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [22]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
#!rm /content/glove.42B.300d.zip

In [23]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [24]:
len(embeding_index['provide'])

300

In [ ]:
# embeding_index['provide']

In [25]:
len(embeding_index)

1917494

In [27]:
X=datafile['question_text']
Y=datafile['target']

In [28]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [29]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [31]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
print(class_weight)

[0.53291767 8.09470577]


In [32]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: np.float64(0.5329176675796012), 1: np.float64(8.094705772985034)}

In [33]:
x_train

,question_text
869608,How can I grow a new YouTube channel in a week?
536471,Which companies in Denver hire international s...
874305,Could Quora be any worse?
610358,What number is missing from the sequence 16 06...
45624,What is rebab like for alcoholics?
...,...
1297808,Why would one pursue MSCS with thesis option?
224067,How do I convince my parents to let me make a ...
30391,Should I file a case against CGC? I deposited ...
1134439,What is something you wished you had known bef...


In [34]:
from nltk import word_tokenize

In [35]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [38]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [40]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
#tk.word_index

In [41]:
x_train.shape

(1044897,)

In [42]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [43]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [44]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [46]:
embedding_matrix.shape

(196015, 300)

In [47]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [48]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 44, 300)   │ 58,804,500 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 44)        │          0 │ text_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 512)       │  1,665,024 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 25)        │      3,225 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 25)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         26 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 60,636,999 (231.31 MB)

 Trainable params: 1,832,499 (6.99 MB)

 Non-trainable params: 58,804,500 (224.32 MB)

In [49]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [50]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [51]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = earlystop)

Epoch 1/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.8489 - loss: 0.3319 

KeyboardInterrupt: 

Note: Due to hardware and time constraints, i had to interrupt the model.fit process after just 1 epoch. However, it still managed to give an excellent roc_auc and accuracy score.

In [52]:
model.save('current_spam_model.h5')
print("Model saved successfully!")

Model saved successfully!


In [55]:
p = model.predict(seq_test_matrix)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 1534s 188ms/step


In [56]:
roc_auc_score(y_test,p)

np.float64(0.9572049612468979)

In [57]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 44, 300)   │ 58,804,500 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 44)        │          0 │ text_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 512)       │  1,665,024 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 25)        │      3,225 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 25)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         26 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 64,301,999 (245.29 MB)

 Trainable params: 1,832,499 (6.99 MB)

 Non-trainable params: 58,804,500 (224.32 MB)

 Optimizer params: 3,665,000 (13.98 MB)

In [58]:
p

array([[0.07144645],
       [0.36721373],
       [0.7632738 ],
       ...,
       [0.19369991],
       [0.00915994],
       [0.08083104]], dtype=float32)

In [59]:
import numpy as np
from sklearn.metrics import *

# Converting probabilities to binary predictions (0 or 1)
y_pred_binary = (p > 0.5).astype(int)

if p.ndim > 1:
    y_pred_binary = y_pred_binary.flatten()
    p_flat = p.flatten()
else:
    p_flat = p

In [60]:
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
roc_auc = roc_auc_score(y_test, p_flat)

In [63]:
print("=== Model Performance Metrics ===")
print(f"Accuracy:           {accuracy:.4f}")
print(f"Precision:          {precision:.4f}")
print(f"Recall:            {recall:.4f}")
print(f"F1-Score:          {f1:.4f}")
print(f"ROC AUC:           {roc_auc:.4f}")

=== Model Performance Metrics ===
Accuracy:           0.8819
Precision:          0.3355
Recall:            0.9150
F1-Score:          0.4910
ROC AUC:           0.9572
